In [1]:
import flet as ft
import nest_asyncio
import numpy as np
from collections import Counter
nest_asyncio.apply()

In [2]:
def main(page: ft.Page):
    
    def exit(e):
        page.window.close()

    def huffman_rate_comp(probabilities, order):
        # sort the probabilities in non-decreasing order
        probabilities = sorted(probabilities)
        total_cost = 0

        while len(probabilities) > 1:
            # we the two smallest probabilities
            p1, p2 = probabilities[0], probabilities[1]
            
            # add the contribution of these two probabilities to the total cost
            total_cost += float(p1) + float(p2)
            
            # Remove the two smallest elements and add their sum to the probabilities list
            probabilities = probabilities[2:]
            probabilities.append(p1 + p2)
            
            # Re-sort the list after the update
            probabilities.sort()

        return total_cost/order

    def entropy_comp(prob):
        # formula for the computation of the entropy
        return -np.sum(prob * np.log2(prob))

    def calculate_entropy_rate(text, order):
        remainder = len(text) % order
        if remainder != 0:
            # adding enough characters to make the length of the text a multiple of the grouping order
            text += ' ' * (order - remainder)  # I decided to use spaces as padding characters

        groups = []
        for i in range(0, len(text), order):
            groups.append(text[i:i+order])

        # I used the Counter() function to count how many occurrences for each group of letters
        counter = Counter(groups)
        
        counts = np.array(list(counter.values()))
        total = np.sum(counts)
        # here we compute the probabilities for each group of letters to occur inside of the text
        probabilities = counts / total

        entropy = entropy_comp(probabilities)
        # to calculate the entropy rate we need to divide the total entropy by the chosen grouping order
        entropy_rate = entropy / order

        # we build a dictionary having as key the group of letters, and as value the probability of that group to occur in the text
        probabilities_dict = {}
        for group,prob in zip(counter.keys(), probabilities):
            probabilities_dict[group] = prob

        return entropy_rate, probabilities_dict

    # function to update the results and called by the compute button
    def computation(e=None):
        text = text_input.value
        # we replace newline characters with a space, as the professor suggested
        text = text.replace("\n", " ")
        order = current_value[0]
        
        # if the user has not digited any text, we ask him to do so with an error message
        if not text:
            error_text.value = "Please enter a text in the field above."
            error_text.color = "red"
            error_text.size = 18
            page.update()
            return 


        error_text.value = ""
        
        entropy_rate, probabilities = calculate_entropy_rate(text, order)
        huffman_rate = huffman_rate_comp(list(probabilities.values()), order)
        
        # write in the GUI the Entropy rate and the Huffman rate
        entropy_rate_label.value = f"Entropy rate = {entropy_rate:.4f} bits/character"
        huffman_rate_label.value = f"Huffman rate = {huffman_rate:.4f} bits/character"
        
        # descending order
        sorted_probs = sorted(probabilities.items(), key=lambda el: el[1], reverse=True)
        
        # update the table with the probabilities of the four most common groups of letters
        table.rows.clear()
        for group, prob in sorted_probs[:4]:
            table.rows.append(ft.DataRow([ft.DataCell(ft.Text('"' + group + '"')), ft.DataCell(ft.Text(f"{prob:.8f}"))]))

        page.update()


    page.title = "Exercise 3 - Part A"
    page.window.width = 850
    page.window.height = 650

    # Create UI elements
    title = ft.Text(
        "Text Entropy Rate Calculator",
        style="headlineLarge",
        weight="bold",
        text_align=ft.TextAlign.CENTER,
    )

    text_input = ft.TextField(
        label="Text",
        multiline=True,
        expand=True,
        height=200,
        value="Sir Isaac Newton (1642–1727) was an English mathematician, physicist, astronomer, and key figure in the scientific revolution. Born in Woolsthorpe, Lincolnshire, Newton is best known for formulating the laws of motion and universal gravitation, which laid the foundation for classical mechanics. His book *Philosophiæ Naturalis Principia Mathematica* (1687), often referred to as the *Principia*, is considered one of the most important works in the history of science. Newton also made significant contributions to mathematics, including the development of calculus (independently of Leibniz), and optics, where he studied the nature of light and color. He served as the president of the Royal Society and held a position as Master of the Royal Mint. Newton’s work transformed the understanding of the physical world and deeply influenced later scientific inquiry."
    )

    current_value = [3]

    def update_textfield():
        grouping_order.value = str(current_value[0])
        page.update()

    def on_decrement(e):
        if current_value[0] > 1:  # Prevent going below 1
            current_value[0] -= 1
            update_textfield()

    def on_increment(e):
        current_value[0] += 1
        update_textfield()

    grouping_order = ft.TextField(
        label="Grouping order",
        value=str(current_value[0]),
        width=100,
        text_align=ft.TextAlign.CENTER,
        read_only=True,  # field non-editable
    )


    decrement_button = ft.IconButton(ft.icons.REMOVE, on_click=on_decrement)
    increment_button = ft.IconButton(ft.icons.ADD, on_click=on_increment)

    grouping_order_row = ft.Row(
            [decrement_button, grouping_order, increment_button],
            alignment=ft.MainAxisAlignment.START,
        )

    entropy_rate_label = ft.Text("Entropy rate = ")
    huffman_rate_label = ft.Text("Huffman rate = ")

    table = ft.DataTable(
        columns=[
            ft.DataColumn(ft.Text("Word")),
            ft.DataColumn(ft.Text("Probability")),
        ],
        rows=[
            ft.DataRow([ft.DataCell(ft.Text('')), ft.DataCell(ft.Text(""))]),
            ft.DataRow([ft.DataCell(ft.Text('')), ft.DataCell(ft.Text(""))]),
            ft.DataRow([ft.DataCell(ft.Text('')), ft.DataCell(ft.Text(""))]),
            ft.DataRow([ft.DataCell(ft.Text('')), ft.DataCell(ft.Text(""))]),
        ],
    )


    # Buttons
    exit_button = ft.ElevatedButton("EXIT", on_click=exit)
    recalculate_button = ft.ElevatedButton("CALCULATE", on_click=computation)

    button_row = ft.Row(controls=[exit_button, recalculate_button], alignment=ft.MainAxisAlignment.START)

    col1 = ft.Column(controls=[grouping_order_row, entropy_rate_label, huffman_rate_label], horizontal_alignment=ft.MainAxisAlignment.START, spacing=40)
    col2 = ft.Column(controls=[table], horizontal_alignment=ft.MainAxisAlignment.END)

    cols_row = ft.Row(controls=[col1, col2], alignment=ft.MainAxisAlignment.SPACE_BETWEEN)

    error_text = ft.Text()

    # Layout the elements
    page.add(
        ft.Row(controls=[title], alignment=ft.MainAxisAlignment.CENTER),
        ft.Row([text_input], alignment=ft.MainAxisAlignment.CENTER),
        ft.Row(controls=[error_text]),
        cols_row,
        button_row
    )

    computation()

    page.update()


In [3]:
# Run the app
# Works only on Windows 11 and macOS. If using Windows 10, comment this line and use the web browser option below.
ft.app(target=main, view=ft.AppView.FLET_APP)

In [ ]:
# Works on all operating systems, including Windows 10. Uncomment this line if using Windows 10 and comment the pop-up window option.
#ft.app(target=main, view=ft.AppView.WEB_BROWSER)